In [69]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
#nltk.download('stopwords')
stop = stopwords.words('english')

In [70]:
df=pd.read_csv("/home/ghost/Jupyter notebook/NLP project/news_dump.csv")
Tags=pd.read_csv("/home/ghost/Jupyter notebook/NLP project/tag_list.csv")

In [71]:
df.dropna(inplace=True)

In [ ]:
 pd.options.display.max_colwidth=100

# Removing hindi data

In [ ]:
from langdetect import detect
df['lang']=df['title'].apply(detect)

In [ ]:
df=df[df['lang']!='hi']

In [ ]:
#convert content column to lower case
df['title']=df['title'].apply(lambda x: x.lower())

In [ ]:
#converting tags in df as list (format=[tag1,tag2,...])
tg=[]
for i in df['tags']:
    tg.append(i.split("|"))

tags=[]
for i in tg:
    tags.extend(i)

tags=pd.unique(tags).tolist()

In [ ]:
for i in range(len(tags)):
    tags[i]=tags[i].lower()

In [ ]:
tg=[[j.lower() for j in i] for i in tg]                    #converting tags in lowercase

In [ ]:
df['tags']=tg

In [ ]:
#split sentence by space
tokenizer = RegexpTokenizer(r'\w+')
df['tokenized_sents'] = df.apply(lambda row: tokenizer.tokenize(row['title']), axis=1)

In [ ]:
#removing stopwords
df['tokenized_sents'] = df['tokenized_sents'].apply(lambda x: [item for item in x if item not in stop])

# remove numbers
df['tokenized_sents'] = df['tokenized_sents'].apply(lambda x: [item for item in x if not item.isdigit()])

#to join tokens together as one sentence
df['tokenized_sents'] = df['tokenized_sents'].apply(lambda x: " ".join(x))

## One Hot Coding

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
MLB=MultiLabelBinarizer(classes=tags)

In [ ]:
OHC_array=MLB.fit_transform(df['tags'])

In [ ]:
OHC=pd.DataFrame(OHC_array,columns=tags)

In [ ]:
print(OHC.shape)

print(df.shape)

In [ ]:
print(df.index[-1])

print(OHC.index[-1])

# both Dataframe has different indexes

In [ ]:
Data=pd.concat([df.set_index(OHC.index), OHC], axis=1, sort=False)   # ".set_index function change or set index of a Dataframe"

In [ ]:
Data.shape

In [ ]:
Data

### Splitting train and test data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(Data['title'],Data.drop(['newsID','title','tokenized_sents','tags','date','media_house','lang'],axis=1),test_size=0.33, random_state=42)

## building model

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
classifier = Pipeline([
('vectorizer', CountVectorizer()),
('tfidf', TfidfTransformer()),
('clf', OneVsRestClassifier(LinearSVC()))])

In [ ]:
classifier.fit(x_train,y_train)

## Predictions and metrics

In [ ]:
predicted = classifier.predict(x_test)

In [ ]:
print(accuracy_score(y_test, predicted))